In [12]:
import json
import numpy as np
import pandas as pd

from pathlib import Path

In [13]:
df = pd.read_json("./results/multinli_transforms_bertscore_dev_matched.jsonl", lines=True).set_index('pairID')
df_annot_labels = pd.read_json("./annotations/gold.json")
df_model_labels = pd.read_json("./results/deberta_large_labels.jsonl", lines=True).set_index('pairID')
df_model_transform_labels = pd.read_json("./results/deberta_large_transform_labels.jsonl", lines=True).set_index('pairID')
df = df.join(df_annot_labels).join(df_model_labels).join(df_model_transform_labels).dropna()
df = df[df['gold_label'] == df['transform_label']]

In [19]:
print(len(df[df['gold_label'] == df['deberta_large_label']]) / len(df))

0.892


In [20]:
print(len(df[df['transform_label'] == df['deberta_large_transform_label']]) / len(df))

0.788


In [26]:
for x, y in df.groupby('genre'):
    print(x)
    print(len(y[y['gold_label'] == y['deberta_large_label']]) / len(y))
    print(len(y[y['transform_label'] == y['deberta_large_transform_label']]) / len(y))

fiction
0.8969072164948454
0.8247422680412371
government
0.9142857142857143
0.8
slate
0.8823529411764706
0.7745098039215687
telephone
0.8712871287128713
0.7623762376237624
travel
0.8947368421052632
0.7789473684210526
